In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os
#from transitions import Machine
import yaml
from threading import Thread
from collections import defaultdict
import numpy as np

In [2]:
%ls -A

.env        .ipynb_checkpoints/  README.md       .wolf_env/
.gitignore  prompts.yml          werewolf.ipynb


In [3]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [4]:
client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

In [28]:
response

ChatCompletion(id='chatcmpl-9C07OBxaMzledSyRuhV4X4bbzdHCe', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The 2020 World Series was played at Globe Life Field in Arlington, Texas.', role='assistant', function_call=None, tool_calls=None))], created=1712647418, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_b28b39ffa8', usage=CompletionUsage(completion_tokens=17, prompt_tokens=53, total_tokens=70))

In [35]:
response.choices[0].message.content

'The 2020 World Series was played at Globe Life Field in Arlington, Texas.'

In [17]:
class ChatSession:
    def __init__(self, debug=False):
        self.system_log = ''
        self.logs = []
        self.day_first_log = [0]
        self.debug = debug
        
    def get_logs_by_day(self, day):
        if len(self.day_first_log) < day:
            raise Exception('Invalid day requested')
        elif self.day_first_log[day-1] == len(self.logs):
            return ['Nothing happened']
        if len(self.day_first_log) == day:
            return self.logs[self.day_first_log[day-1]:]
        return self.logs[self.day_first_log[day-1]:self.day_first_log[day]]
        
    def new_day(self):
        self.day_first_log.append(len(self.logs))

    def history(self, n_recent=500):
        window = self.logs
        if len(window) > n_recent:
            window = window[-500:]
        return '\n'.join([log for log in window])

    def add_log(self, log):
        self.logs.append(log)

    def ask_until_keyword(self, query, keyword, n_attempts=2):
        index = -1
        n = 0
        while index == -1 and n < n_attempts:
            n += 1
            result = self.query(query)
            index = result.find(keyword)
        if index == -1:
            raise Exception(f"Couldn't get the keyword, query: {query}")
        return result, index
    
    def query(self, query):
        message = self.history() + '\n\n' + query
        response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
            {"role": "system", "content": self.system_log}
            {"role": "user", "content": message},
          ]
        )
        if self.debug:
            print("\nQUERY:")
            print(message)
            print("REPLY:")
            print(response.choices[0].message.content)
            print('\n')
        return response.choices[0].message.content

class PlayerRelationship:
    def __init__(self, self_role, other_player, backward_rel=None):
        self.status = "not sure"
        self.self_status = self_role
        self.other_player = other_player
        if backward_rel is not None:
            self.value = np.random.randint(-2, 3) + backward_rel.value
            self.value = np.clip(self.value, -10, 10)
        else:
            self.value = np.random.randint(-10, 11)
    
    def change(self, value):
        self.value = np.clip(self.value+value, -10, 10)
    
    def reveal(self, new_status):
        self.status = new_status
        if self.self_status != "wolf" and new_status == "wolf":
            self.change(-15)
            
    def other_player_short_description(self):
        result = prompts[f'player_short_description']
        description = prompts[f'player_short_{self.other_player.name}']
        result = result.replace('<NAME>', self.other_player.name)
        result = result.replace('<DESCRIPTION>', description)
        result = result.replace('<RELATIONSHIP>', self.describe())
        return result
    
    def others_role_description(self):
        result = self.status
        if not self.other_player.is_alive():
            result = "dead " + result
        if self.self_status == self.status:
            result = "fellow " + result
        return result
    
    def describe(self):
        return f"Role: {self.others_role_description()}, how much I like them: {self.value}"
    
class Player:
    roles = {"human", "wolf", "seer"}
    
    def __init__(self, role, name):
        if role not in Player.roles:
            raise ValueError(f"Invalid role: {role}")
        self.name = name
        self.role = role
        self.alive = True
        self.relationships = {}
        self.day = 1
        self.main_session = ChatSession()
        self.reflection_session = ChatSession(debug=self.name=='Bella')
        self.spoken_last_round = False
        
    def namify(self, name):
        return name[0].upper() + name[1:].lower()

    def add_message(self, message):
        self.main_session.add_log(message)
        
    def add_reflection(self, message):
        self.reflection_session.add_log(message)
    
    def get_summary(self):
        prompt_prefix = self.todays_discussion()
        return self.reflection_session.query(prompt_prefix + prompts['player_summarize'])
    
    def init_main_session(self, player_names, wolf_names):
        intro_prompt = prompts[f'player_intro']
        intro_prompt = intro_prompt.replace('<PLAYERS>', player_names)
        intro_prompt = intro_prompt.replace('<DESCRIPTION>', self.name + '.\n' + prompts[f'player_{self.name}'])
        intro_prompt += prompts[f'player_role_{self.role}'].replace('<WOLFS>', wolf_names)
        intro_prompt += "\nKnowledge on other players:\n" + ''.join([rel.other_player_short_description() for rel in self.relationships.values()])
        self.reflection_session.system_log = intro_prompt

    def is_wolf(self):
        return self.role == "wolf"

    def is_alive(self):
        return self.alive

    def die(self):
        self.alive = False
        
    def strategize(self, add=True):
        reflection = self.reflection_session.query(prompts[f'player_strategize_{self.role}'])
        reflection = f"My thoughts on what's happening so far: {reflection}\n--------"
        if add:
            self.add_reflection(reflection)
        return reflection

    def new_day(self):
        self.reflection_session.add_log(f'Day {self.day} summary:\n {self.get_summary()}\n--------\n')
        self.strategize()
        self.main_session.new_day()
        self.reflection_session.new_day()
        self.day += 1
        
    def todays_discussion(self):
        return_sym = '\n'
        return f"Today's discussion so far: {return_sym.join(self.main_session.get_logs_by_day(self.day))}\n"
    
    def rel_status(self):
        result = ""
        for name, rel in self.relationships.items():
            result += name + " - " + rel.describe() + "\n"
        return result
    
    def query_reflection_prefix(self):
        prompt_prefix = self.todays_discussion()
        prompt_prefix += self.rel_status()
        return prompt_prefix

    def vote(self, eligible_targets):
        prompt_prefix = self.query_reflection_prefix()
        eligible_targets = ', '.join(eligible_targets)
        prompt = f"{prompt_prefix}\n-------\n{prompts['player_vote_reflect'].replace('<TARGETS>', eligible_targets)}"
        reflection, index = self.reflection_session.ask_until_keyword(query=prompt, keyword='VOTE')
        target_name = reflection[index:].split()[1].strip(' \t\n.,;')
        return self.namify(target_name)

    def speak_up(self):
        prompt_prefix = self.query_reflection_prefix()
        if self.day > 1:
            reflection = self.strategize(add=False)
            prompt_prefix += reflection
        reply = self.reflection_session.query(prompt_prefix + prompts[f'player_participate_{self.role}'])
        return reply

    def murder(self):
        prompt_prefix = self.query_reflection_prefix()
        prompt = f"{prompt_prefix}\n-------\n{prompts['player_murder_reflect']}"
        reflection, index = self.reflection_session.ask_until_keyword(query=prompt, keyword='ELIMINATE')
        target_name = reflection[index:].split()[1].strip(' \t\n.,;')
        return self.namify(target_name)

    def inspect(self):
        prompt_prefix = self.todays_discussion()
        prompt = f"{prompt_prefix}\n-------\n{prompts['player_seer_reflect']}"
        reflection, index = self.reflection_session.ask_until_keyword(query=prompt, keyword='INSPECT')
        target_name = reflection[index:].split()[1].strip(' \t\n.,;')
        return self.namify(target_name)
        


class Game:
    def __init__(self):
        self.players = {}
    
    def init_relationships(self, new_player):
        for player in self.players.values():
            if player is not new_player:
                player.relationships[new_player.name] = PlayerRelationship(player.role, new_player)
                new_player.relationships[player.name] = PlayerRelationship(new_player.role, player)
    
    def add_player(self, name, role):
        new_player = Player(role, name)
        self.players[name] = new_player
        self.init_relationships(new_player)

    def get_victor(self):
        wolfs = 0
        humans = 0
        for player in self.players.values():
            if player.is_alive():
                if player.is_wolf():
                    wolfs += 1
                else:
                    humans += 1
        if wolfs == 0:
            return "humans"
        elif wolfs >= humans:
            return "wolfs"
        else:
            return "not_yet"
        
    def communicate_roles(self):
        wolfs = [player.name for player in self.players.values() if player.is_wolf()]
        wolfs = ', '.join(wolfs)
        player_names = [player.name for player in self.players.values()]
        player_names = ', '.join(player_names)
        for player in self.players.values():
            player.init_main_session(player_names=player_names, wolf_names=wolfs)

    def vote(self):
        for player in self.players.values():
            player.add_message(prompts['mod_vote'])
        
        eligible_targets = [player.name for player in self.players.values() if player.is_alive()]
        votes = defaultdict(lambda : 0)
        vote_strings = []
        for player in self.players.values():
            target = player.vote(eligible_targets)
            votes[target] += 1
            vote_strings.append(f"{player.name} voted for {target}")

        vote_string = ', '.join(vote_strings)
        unalived = max(votes, key=votes.get)

        print('VOTES:')
        print(votes)
        print(unalived)
        if not unalived in self.players or not self.players[unalived].is_alive():
            raise Exception("Teach them to vote better?")
        
        self.players[unalived].die()
        
        message = prompts['mod_vote_killed']
        message = message.replace('<NAME>', unalived)
        message = message.replace('<VOTE>', vote_string)
        self.message(message)

    def night(self):
        wolfs = [player for player in self.players.values() if player.is_wolf() and player.is_alive()]
        
        murdered = None
        if len(wolfs) > 0:
            killer = wolfs[np.random.randint(len(wolfs))]
            murdered = killer.murder()
            print(f'MURDER TARGET: "{murdered}"')
        if not murdered in self.players:
            murdered = None
            
        if murdered is None:
            message = prompts['mod_new_day_quiet']
        else:
            self.players[murdered].die()
            message = prompts['mod_new_day_murdered'].replace('<NAME>', murdered)
        self.message(message)
        

    def new_day(self):
        for player in self.players.values():
            player.new_day()

    def speak(self, player, message):
        for cur_player in self.players.values():
            if cur_player is player:
                cur_player.add_message(f'You say: "{message}"')
            else:
                cur_player.add_message(f'{player.name} says: "{message}"')

    def message(self, message):
        for player in self.players.values():
            player.add_message(message)
    
    def seer_inspect(self):
        players = [player for player in self.players.values() if player.is_alive()]
        for player in players:
            if player.role == "seer" and player.is_alive():
                target = player.inspect()
                investigation = f"You attempt to investigate {target}.\n"
                if target in self.players and self.players[target].is_alive():
                    investigation += f"{target}'s role is {self.players[target].role} (only you know that for now)"
                    player.relationships[target].reveal(self.players[target].role)
                else:
                    investigation += f"The spirits did not answer (target is dead/wrong name?)"
                player.add_reflection(investigation)
                break

    def day_loop(self):
        players = [player for player in self.players.values() if player.is_alive()]
        speak_requests = []
        for i in range(1):
            for player in players:
                text = player.speak_up()
                self.speak(player, text)
                player.spoken_last_round = True
        self.vote()
        self.seer_inspect()
        return True
                
    def loop(self):
        while self.get_victor() == "not_yet":
            print('New day')
            self.day_loop()
            if self.get_victor() == "not_yet":
                self.night()
            self.new_day()

                
with open('prompts.yml', 'r') as f:
    prompts = yaml.safe_load(f)

game = Game()
game.add_player('Alex', 'human')
game.add_player('Bella', 'seer')
game.add_player('Charlie', 'human')
game.add_player('Diana', 'wolf')
game.add_player('Ethan', 'human')
game.add_player('Fiona', 'human')
game.add_player('Gabriel', 'wolf')
game.add_player('Hannah', 'human')
game.add_player('Isaac', 'human')
game.add_player('Jasmine', 'human')
game.add_player('Kieran', 'human')
game.communicate_roles()
game.loop()

New day

QUERY:
You act a character that finds himself in a werewolf game scenario. You are in company of other people: Alex, Bella, Charlie, Diana, Ethan, Fiona, Gabriel, Hannah, Isaac, Jasmine, Kieran. Some of you become bloothirsty wolfs in human disguise, and every night the wolfs will attempt to murder one person. All of you will vote to hang one person every day. During the day you shall discuss who to vote for hanging, every person shall speak before the vote starts.
You also have a Seer among you, who can inspect one person at night to find out whether they are a wolf or not.
You are Bella.
Bella's heart belongs to the vast landscapes she paints, inspired by the village and its surrounding nature. Raised by the village's most renowned artist, Bella developed a deep appreciation for beauty in all its forms. She is gentle, empathetic, and has a knack for seeing the good in people, which makes her a beloved figure among the villagers. However, her sensitivity also means she takes 

VOTES:
defaultdict(<function Game.vote.<locals>.<lambda> at 0x7fe1aa4cf4c0>, {'Isaac': 5, 'Gabriel': 2, 'Fiona': 3, 'Kieran': 1})
Isaac

QUERY:
You act a character that finds himself in a werewolf game scenario. You are in company of other people: Alex, Bella, Charlie, Diana, Ethan, Fiona, Gabriel, Hannah, Isaac, Jasmine, Kieran. Some of you become bloothirsty wolfs in human disguise, and every night the wolfs will attempt to murder one person. All of you will vote to hang one person every day. During the day you shall discuss who to vote for hanging, every person shall speak before the vote starts.
You also have a Seer among you, who can inspect one person at night to find out whether they are a wolf or not.
You are Bella.
Bella's heart belongs to the vast landscapes she paints, inspired by the village and its surrounding nature. Raised by the village's most renowned artist, Bella developed a deep appreciation for beauty in all its forms. She is gentle, empathetic, and has a knack for

MURDER TARGET: "Fiona"

QUERY:
You act a character that finds himself in a werewolf game scenario. You are in company of other people: Alex, Bella, Charlie, Diana, Ethan, Fiona, Gabriel, Hannah, Isaac, Jasmine, Kieran. Some of you become bloothirsty wolfs in human disguise, and every night the wolfs will attempt to murder one person. All of you will vote to hang one person every day. During the day you shall discuss who to vote for hanging, every person shall speak before the vote starts.
You also have a Seer among you, who can inspect one person at night to find out whether they are a wolf or not.
You are Bella.
Bella's heart belongs to the vast landscapes she paints, inspired by the village and its surrounding nature. Raised by the village's most renowned artist, Bella developed a deep appreciation for beauty in all its forms. She is gentle, empathetic, and has a knack for seeing the good in people, which makes her a beloved figure among the villagers. However, her sensitivity also m


QUERY:
You act a character that finds himself in a werewolf game scenario. You are in company of other people: Alex, Bella, Charlie, Diana, Ethan, Fiona, Gabriel, Hannah, Isaac, Jasmine, Kieran. Some of you become bloothirsty wolfs in human disguise, and every night the wolfs will attempt to murder one person. All of you will vote to hang one person every day. During the day you shall discuss who to vote for hanging, every person shall speak before the vote starts.
You also have a Seer among you, who can inspect one person at night to find out whether they are a wolf or not.
You are Bella.
Bella's heart belongs to the vast landscapes she paints, inspired by the village and its surrounding nature. Raised by the village's most renowned artist, Bella developed a deep appreciation for beauty in all its forms. She is gentle, empathetic, and has a knack for seeing the good in people, which makes her a beloved figure among the villagers. However, her sensitivity also means she takes the vill


QUERY:
You act a character that finds himself in a werewolf game scenario. You are in company of other people: Alex, Bella, Charlie, Diana, Ethan, Fiona, Gabriel, Hannah, Isaac, Jasmine, Kieran. Some of you become bloothirsty wolfs in human disguise, and every night the wolfs will attempt to murder one person. All of you will vote to hang one person every day. During the day you shall discuss who to vote for hanging, every person shall speak before the vote starts.
You also have a Seer among you, who can inspect one person at night to find out whether they are a wolf or not.
You are Bella.
Bella's heart belongs to the vast landscapes she paints, inspired by the village and its surrounding nature. Raised by the village's most renowned artist, Bella developed a deep appreciation for beauty in all its forms. She is gentle, empathetic, and has a knack for seeing the good in people, which makes her a beloved figure among the villagers. However, her sensitivity also means she takes the vill

...


QUERY:
You act a character that finds himself in a werewolf game scenario. You are in company of other people: Alex, Bella, Charlie, Diana, Ethan, Fiona, Gabriel, Hannah, Isaac, Jasmine, Kieran. Some of you become bloothirsty wolfs in human disguise, and every night the wolfs will attempt to murder one person. All of you will vote to hang one person every day. During the day you shall discuss who to vote for hanging, every person shall speak before the vote starts.
You also have a Seer among you, who can inspect one person at night to find out whether they are a wolf or not.
You are Bella.
Bella's heart belongs to the vast landscapes she paints, inspired by the village and its surrounding nature. Raised by the village's most renowned artist, Bella developed a deep appreciation for beauty in all its forms. She is gentle, empathetic, and has a knack for seeing the good in people, which makes her a beloved figure among the villagers. However, her sensitivity also means she takes the vill


QUERY:
You act a character that finds himself in a werewolf game scenario. You are in company of other people: Alex, Bella, Charlie, Diana, Ethan, Fiona, Gabriel, Hannah, Isaac, Jasmine, Kieran. Some of you become bloothirsty wolfs in human disguise, and every night the wolfs will attempt to murder one person. All of you will vote to hang one person every day. During the day you shall discuss who to vote for hanging, every person shall speak before the vote starts.
You also have a Seer among you, who can inspect one person at night to find out whether they are a wolf or not.
You are Bella.
Bella's heart belongs to the vast landscapes she paints, inspired by the village and its surrounding nature. Raised by the village's most renowned artist, Bella developed a deep appreciation for beauty in all its forms. She is gentle, empathetic, and has a knack for seeing the good in people, which makes her a beloved figure among the villagers. However, her sensitivity also means she takes the vill

VOTES:
defaultdict(<function Game.vote.<locals>.<lambda> at 0x7fe1aa961440>, {'Diana': 11})
Diana

QUERY:
You act a character that finds himself in a werewolf game scenario. You are in company of other people: Alex, Bella, Charlie, Diana, Ethan, Fiona, Gabriel, Hannah, Isaac, Jasmine, Kieran. Some of you become bloothirsty wolfs in human disguise, and every night the wolfs will attempt to murder one person. All of you will vote to hang one person every day. During the day you shall discuss who to vote for hanging, every person shall speak before the vote starts.
You also have a Seer among you, who can inspect one person at night to find out whether they are a wolf or not.
You are Bella.
Bella's heart belongs to the vast landscapes she paints, inspired by the village and its surrounding nature. Raised by the village's most renowned artist, Bella developed a deep appreciation for beauty in all its forms. She is gentle, empathetic, and has a knack for seeing the good in people, which make


QUERY:
You act a character that finds himself in a werewolf game scenario. You are in company of other people: Alex, Bella, Charlie, Diana, Ethan, Fiona, Gabriel, Hannah, Isaac, Jasmine, Kieran. Some of you become bloothirsty wolfs in human disguise, and every night the wolfs will attempt to murder one person. All of you will vote to hang one person every day. During the day you shall discuss who to vote for hanging, every person shall speak before the vote starts.
You also have a Seer among you, who can inspect one person at night to find out whether they are a wolf or not.
You are Bella.
Bella's heart belongs to the vast landscapes she paints, inspired by the village and its surrounding nature. Raised by the village's most renowned artist, Bella developed a deep appreciation for beauty in all its forms. She is gentle, empathetic, and has a knack for seeing the good in people, which makes her a beloved figure among the villagers. However, her sensitivity also means she takes the vill


QUERY:
You act a character that finds himself in a werewolf game scenario. You are in company of other people: Alex, Bella, Charlie, Diana, Ethan, Fiona, Gabriel, Hannah, Isaac, Jasmine, Kieran. Some of you become bloothirsty wolfs in human disguise, and every night the wolfs will attempt to murder one person. All of you will vote to hang one person every day. During the day you shall discuss who to vote for hanging, every person shall speak before the vote starts.
You also have a Seer among you, who can inspect one person at night to find out whether they are a wolf or not.
You are Bella.
Bella's heart belongs to the vast landscapes she paints, inspired by the village and its surrounding nature. Raised by the village's most renowned artist, Bella developed a deep appreciation for beauty in all its forms. She is gentle, empathetic, and has a knack for seeing the good in people, which makes her a beloved figure among the villagers. However, her sensitivity also means she takes the vill

In [ ]:
'"la la"'.strip('" \n')

In [18]:
game.get_victor()

'humans'

In [14]:
game.players['Diana'].is_alive()

True

In [19]:
print('\n'.join(game.players['Bella'].main_session.logs))

Alex says: "My friends, it's clear that we need to start discussing and making decisions on who we suspect might be the wolves among us. Based on prior interactions and observations, I have my suspicions about a few individuals. Let's not waste any time and start voicing our thoughts and concerns openly for the good of our village. The longer we wait, the more lives are at risk. Let's work together to uncover the truth and protect one another."
You say: "My friends, I believe it's crucial for us to consider the actions and behaviors of each individual within our village. Ethan's charismatic nature may mask hidden motives, while Gabriel's mysterious aura raises suspicion. I urge you all to carefully observe and question each other to ensure the safety of our community. Let us trust our instincts and work together to uncover the true identities of those who seek to harm us."
Charlie says: "My friends, as the blacksmith's apprentice, I have always had a keen eye for detail and observation

In [20]:
print('\n'.join(game.players['Diana'].reflection_session.logs))

You act a character that finds himself in a werewolf game scenario. You are in company of other people: Alex, Bella, Charlie, Diana, Ethan, Fiona, Gabriel, Hannah, Isaac, Jasmine, Kieran. Some of you become bloothirsty wolfs in human disguise, and every night the wolfs will attempt to murder one person. All of you will vote to hang one person every day. During the day you shall discuss who to vote for hanging, every person shall speak before the vote starts.
You also have a Seer among you, who can inspect one person at night to find out whether they are a wolf or not.
You are Diana.
Diana, the librarian, is a keeper of knowledge and stories. Her love for books was kindled in her early years, leading her to take charge of the village library. Diana is wise beyond her years, with an air of mystery that surrounds her. She believes in the power of knowledge and is often found with her nose in a book, uncovering ancient lore and forgotten histories that she believes hold the key to understa